In [31]:
import sys
import warnings
sys.path.append("../Pythonscripts")
sys.path.append(r"C:\Users\atedi\OneDrive\Documents\SageBionetworks\synapse\Scripts")
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import synapseclient as sc
from pdkit_feature_utils import pdkit_pipeline
import utils
from utils import get_synapse_table, get_acceleration_ts, open_filepath
from sfm_feature_utils import get_spectrum
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pdkit
import datetime
from datetime import datetime
%matplotlib inline
from dateutil import parser
warnings.simplefilter("ignore")

In [19]:
syn = sc.login()

Welcome, aryton tediarjo!



INFO:synapseclient_default:Welcome, aryton tediarjo!



In [21]:
android_data = get_synapse_table(syn, ["4b1faed8-9f46-457d-b3cc-e158c2af4f65",
                              "ae8fe177-8e0f-4cd7-a9eb-bd5a82ac02d3",
                              "34b5cc66-2913-48e2-be04-07ebade1300d",
                              "03366c61-0e20-4afd-973c-4f0a5685385d",
                              "ea790311-50de-4991-b9a6-f1026531d19a",
                              "e3b19543-41dc-402e-aafe-c2c50b469be4",
                            "6d9b0436-6740-4c50-81ea-4696647b49fb"], "syn12514611", "V2")

Querying V2 Data


INFO:synapseclient_default:Downloading 0 files, 218 cached locally


In [33]:
data = open_filepath(android_data["walk_motion.json_pathfile"].iloc[0]).sort_values(by = "timestamp")

In [35]:
utils.clean_accelerometer_data(data)

,eventAccuracy,sensor,sensorType,timestamp,timestampDate,unit,uptime,x,y,z,td,AA
time,,,,,,,,,,,,
1970-01-01 00:00:00.000000000,0,NaN,rotationRate,0.000000,NaN,rad/s,312139.436357,0.579876,-0.016732,-0.228254,0.000000,0.623407
1970-01-01 00:00:00.000000000,0,NaN,acceleration,0.000000,NaN,g,312139.436357,-0.315049,-0.826210,-0.017828,0.000000,0.884419
1970-01-01 00:00:00.019708417,0,NaN,acceleration,0.019708,NaN,g,312139.456065,-0.577346,-0.882626,-0.077175,0.019708,1.057503
1970-01-01 00:00:00.039426697,0,NaN,acceleration,0.039427,NaN,g,312139.475784,-0.399062,-0.794461,0.042007,0.039427,0.890047
1970-01-01 00:00:00.059135114,0,NaN,acceleration,0.059135,NaN,g,312139.495492,-0.354613,-0.876276,0.220046,0.059135,0.970582
1970-01-01 00:00:00.078843531,0,NaN,acceleration,0.078844,NaN,g,312139.515200,-0.433498,-0.968348,0.323841,0.078844,1.109275
1970-01-01 00:00:00.098552073,0,NaN,acceleration,0.098552,NaN,g,312139.534909,-0.266204,-0.925365,0.237142,0.098552,0.991666
1970-01-01 00:00:00.118260490,0,NaN,acceleration,0.118260,NaN,g,312139.554617,-0.317980,-0.885801,0.131148,0.118260,0.950239
1970-01-01 00:00:00.137968907,0,NaN,acceleration,0.137969,NaN,g,312139.574326,-0.603722,-1.088262,0.008059,0.137969,1.244532


# Android Datetime

In [7]:
# convert zulu to acceptable 
def convertZuluTimeStamp(params):
    timestamp = open_filepath(params)["timestampDate"].iloc[0]
    return parser.parse(timestamp)

def isSensorEventsEqual(params):
    try:
        sensor_event_uptime = open_filepath(params)["uptime"].iloc[0]
        accel_start_uptime = open_filepath(params)["uptime"].iloc[1]

        if sensor_event_uptime == gyro_start_uptime == accel_start_uptime:
            return True
        else:
            return False
    except:
        return False

def timeDiffSign(createdOn, timeStampDate):
    if createdOn > timeStampDate:
        return "positive"
    else:
        return "negative"

In [8]:
android_data["timeStampDate"] = pd.to_datetime(android_data["walk_motion.json_pathfile"] \
                                               .apply(convertZuluTimeStamp), utc = True)
android_data["createdOn"] = pd.DatetimeIndex(android_data["createdOn"]).tz_localize("UTC")
android_data["time_diff"] = pd.to_datetime(android_data["createdOn"] , utc = True) \
                                - android_data["timeStampDate"]
android_data["time_diff_sign"] = android_data.apply(lambda x: timeDiffSign(x["createdOn"], x["timeStampDate"]), axis = 1)

In [15]:
test = android_data[["healthCode", "createdOn", "createdOnTimeZone", 
              "uploadDate", "timeStampDate", "time_diff", "time_diff_sign"]].set_index("createdOn")

In [17]:
test[test["healthCode"] == "6d9b0436-6740-4c50-81ea-4696647b49fb"]

,healthCode,createdOnTimeZone,uploadDate,timeStampDate,time_diff,time_diff_sign
createdOn,,,,,,
2019-09-02 21:28:38.415000+00:00,6d9b0436-6740-4c50-81ea-4696647b49fb,-700,2019-09-03,2019-08-26 20:06:00.676610+00:00,7 days 01:22:37.738390,positive
2019-09-03 05:53:38.551000+00:00,6d9b0436-6740-4c50-81ea-4696647b49fb,-700,2019-09-03,2019-08-26 20:06:00.675783+00:00,7 days 09:47:37.875217,positive
2019-09-06 21:39:26.484000+00:00,6d9b0436-6740-4c50-81ea-4696647b49fb,-700,2019-09-07,2019-08-26 20:06:00.676079+00:00,11 days 01:33:25.807921,positive
2019-09-26 16:48:33.982000+00:00,6d9b0436-6740-4c50-81ea-4696647b49fb,-700,2019-09-27,2019-09-26 16:33:33.322248+00:00,0 days 00:15:00.659752,positive
2019-09-26 16:50:10.523000+00:00,6d9b0436-6740-4c50-81ea-4696647b49fb,-700,2019-09-27,2019-09-26 16:33:33.323069+00:00,0 days 00:16:37.199931,positive
2019-10-22 06:22:37.588000+00:00,6d9b0436-6740-4c50-81ea-4696647b49fb,-700,2019-10-22,2019-10-21 22:46:38.262158+00:00,0 days 07:35:59.325842,positive


In [41]:
ios_data = get_synapse_table(syn, ["d2c2a238-36ab-493e-9728-589eb0797fb3",
                                        "8fd8f4c8-6b47-45f0-9a0c-58b1c6883b26",
                              "debb8d54-435c-44ff-8684-5bc05cc407b7",
                              "e1882f09-06ae-4cf0-8a1d-3e8a6562d54b",
                              "1fa674e9-2c49-4260-83f0-681ace2a5125",
                              "d2c2a238-36ab-493e-9728-589eb0797fb3"], "syn12514611", "V2")

Querying V2 Data


INFO:synapseclient_default:Downloading 0 files, 464 cached locally


In [43]:
ios_data["timeStampDate"] = pd.to_datetime(ios_data["walk_motion.json_pathfile"] \
                                               .apply(convertZuluTimeStamp), utc = True)
ios_data["time_diff"] = pd.to_datetime(ios_data["createdOn"] , utc = True) - ios_data["timeStampDate"]

In [56]:
temp = utils.clean_accelerometer_data(open_filepath(ios_data["walk_motion.json_pathfile"].iloc[0]))

In [58]:
all(temp.index[:-1] <= temp.index[1:])

True

In [60]:
temp.sort_index()

,eventAccuracy,referenceCoordinate,sensorType,stepPath,timestamp,timestampDate,uptime,w,x,y,z,td,AA
time,,,,,,,,,,,,,
1970-01-01 00:00:00.000000000,NaN,NaN,accelerometer,WalkAndBalance/WalkAndBalance/walk/motion,0.032473,NaN,179440.526245,NaN,0.333160,-0.947479,0.009872,0.000000,1.004395
1970-01-01 00:00:00.010062001,NaN,NaN,accelerometer,WalkAndBalance/WalkAndBalance/walk/motion,0.042535,NaN,179440.536307,NaN,0.336502,-0.955399,0.002716,0.010062,1.012930
1970-01-01 00:00:00.020123001,NaN,NaN,accelerometer,WalkAndBalance/WalkAndBalance/walk/motion,0.052596,NaN,179440.546368,NaN,0.343063,-0.957443,-0.002823,0.020123,1.017054
1970-01-01 00:00:00.030182001,NaN,NaN,accelerometer,WalkAndBalance/WalkAndBalance/walk/motion,0.062655,NaN,179440.556427,NaN,0.349304,-0.958801,-0.007706,0.030182,1.020477
1970-01-01 00:00:00.033962001,NaN,NaN,gyro,WalkAndBalance/WalkAndBalance/walk/motion,0.066435,NaN,179440.560207,NaN,0.030949,0.032580,0.054879,0.033962,0.070929
1970-01-01 00:00:00.040242001,NaN,NaN,accelerometer,WalkAndBalance/WalkAndBalance/walk/motion,0.072715,NaN,179440.566487,NaN,0.357025,-0.953537,-0.010925,0.040242,1.018243
1970-01-01 00:00:00.044022001,NaN,NaN,gyro,WalkAndBalance/WalkAndBalance/walk/motion,0.076495,NaN,179440.570267,NaN,0.063333,-0.023882,0.031578,0.044022,0.074690
1970-01-01 00:00:00.050302001,NaN,NaN,accelerometer,WalkAndBalance/WalkAndBalance/walk/motion,0.082775,NaN,179440.576547,NaN,0.359482,-0.946198,-0.013824,0.050302,1.012279
1970-01-01 00:00:00.054082001,NaN,NaN,gyro,WalkAndBalance/WalkAndBalance/walk/motion,0.086555,NaN,179440.580327,NaN,0.096080,-0.056356,0.073670,0.054082,0.133547


In [18]:
ios_data[["healthCode", "createdOn", "createdOnTimeZone", 
              "uploadDate", "timeStampDate", "time_diff", "time_diff_sign"]]

,healthCode,createdOn,createdOnTimeZone,uploadDate,timeStampDate,time_diff,time_diff_sign
0,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-10 22:22:39.470000+00:00,-400,2019-08-12,2019-08-10 22:17:22.926000+00:00,00:05:16.544000,positive
1,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-12 20:13:18.094000+00:00,-400,2019-08-13,2019-08-12 20:10:43.430000+00:00,00:02:34.664000,positive
2,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-12 20:25:23.322000+00:00,-400,2019-08-13,2019-08-12 20:24:09.629000+00:00,00:01:13.693000,positive
3,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-13 11:16:08.942000+00:00,-400,2019-08-14,2019-08-13 10:32:17.045000+00:00,00:43:51.897000,positive
4,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-14 10:29:52.271000+00:00,-400,2019-08-15,2019-08-14 10:28:12.012000+00:00,00:01:40.259000,positive
5,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-15 21:04:11.508000+00:00,-400,2019-08-16,2019-08-15 21:03:10.739000+00:00,00:01:00.769000,positive
6,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-16 09:54:02.257000+00:00,-400,2019-08-17,2019-08-16 09:52:43.630000+00:00,00:01:18.627000,positive
7,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-16 14:38:50.293000+00:00,-400,2019-08-17,2019-08-16 14:37:49.563000+00:00,00:01:00.730000,positive
8,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-17 18:25:25.455000+00:00,-400,2019-08-18,2019-08-17 18:24:24.730000+00:00,00:01:00.725000,positive
9,1fa674e9-2c49-4260-83f0-681ace2a5125,2019-08-19 11:02:43.238000+00:00,-400,2019-08-20,2019-08-19 11:01:32.853000+00:00,00:01:10.385000,positive


In [ ]:
ios_data[["healthCode", "createdOn", "createdOnTimeZone", 
              "uploadDate", "timeStampDate", "time_diff", "time_diff_sign"]]

In [101]:
ios_data["time_diff"] = pd.to_datetime(ios_data["createdOn"] , utc = True) - ios_data["timeStampDate"]


In [107]:
open_filepath(android_data["walk_motion.json_pathfile"].iloc[0])

,eventAccuracy,sensor,sensorType,timestamp,timestampDate,unit,uptime,x,y,z
0,0,"{'mFifoMaxEventCount': 3585, 'mFifoReservedEve...",rotationRate,0.000000,2019-04-13T23:41:46.463829981Z,NaN,312139.436357,NaN,NaN,NaN
1,0,NaN,rotationRate,0.000000,NaN,rad/s,312139.436357,0.579876,-0.016732,-0.228254
2,0,NaN,acceleration,0.000000,NaN,g,312139.436357,-0.315049,-0.826210,-0.017828
3,0,NaN,acceleration,0.019708,NaN,g,312139.456065,-0.577346,-0.882626,-0.077175
4,0,NaN,acceleration,0.039427,NaN,g,312139.475784,-0.399062,-0.794461,0.042007
5,0,NaN,acceleration,0.059135,NaN,g,312139.495492,-0.354613,-0.876276,0.220046
6,0,NaN,acceleration,0.078844,NaN,g,312139.515200,-0.433498,-0.968348,0.323841
7,0,NaN,acceleration,0.098552,NaN,g,312139.534909,-0.266204,-0.925365,0.237142
8,0,NaN,acceleration,0.118260,NaN,g,312139.554617,-0.317980,-0.885801,0.131148
9,0,NaN,acceleration,0.137969,NaN,g,312139.574326,-0.603722,-1.088262,0.008059


In [23]:
import ast
def map_to_json(params):
    params  = ast.literal_eval(params)
    if isinstance(params, dict):
        return params
    else:
        return np.NaN
    
def normalize_feature(data, feature):
    normalized_data = data[feature].map(map_to_json) \
                                .apply(pd.Series) \
                                .fillna("#ERROR").add_prefix('{}.'.format(feature))
    data = pd.concat([data, normalized_data], 
                     axis = 1).drop(feature, axis = 1)
    return data

In [24]:
data = pd.read_csv("../PythonScripts/test.csv", index_col = [0])

In [26]:
data = normalize_feature(data, "walk_motion.json_features_x")
data = normalize_feature(data, "walk_motion.json_features_y")
data = normalize_feature(data, "walk_motion.json_features_z")
data = normalize_feature(data, "walk_motion.json_features_AA")

In [28]:
data.columns

Index(['recordId', 'phoneInfo', 'createdOn', 'healthCode',
       'walk_motion.json_path_id', 'balance_motion.json_path_id',
       'walk_motion.json_pathfile', 'balance_motion.json_pathfile',
       'walk_motion.json_features_x.no_of_steps',
       'walk_motion.json_features_x.mean_freeze_index',
       'walk_motion.json_features_x.median_freeze_index',
       'walk_motion.json_features_x.max_freeze_index',
       'walk_motion.json_features_x.count_freeze_index',
       'walk_motion.json_features_x.speed_of_gait',
       'walk_motion.json_features_x.gait_step_regularity',
       'walk_motion.json_features_x.gait_stride_regularity',
       'walk_motion.json_features_x.gait_symmetry',
       'walk_motion.json_features_x.frequency_of_peaks',
       'walk_motion.json_features_y.no_of_steps',
       'walk_motion.json_features_y.mean_freeze_index',
       'walk_motion.json_features_y.median_freeze_index',
       'walk_motion.json_features_y.max_freeze_index',
       'walk_motion.json_featur

In [71]:
test.join(json_normalize(test["sensor"].map(json.loads)).drop("sensor", axis = 1))

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [58]:
var = "AA"
gp = pdkit.GaitProcessor(duration=data.td[-1])
data = gp.resample_signal(data)
    ### instantiate empty dictionary ###
feature_dict = {}
try:  
    no_of_steps = gp.gait(data[var])[0]
except:
    no_of_steps = 0
try:
    gait_step_regularity, gait_stride_regularity, gait_symmetry = gp.gait_regularity_symmetry(data[var])
except:
    gait_step_regularity = 0
    gait_stride_regularity = 0
    gait_symmetry = 0
try:
    frequency_of_peaks = gp.frequency_of_peaks(data[var])
except:
    frequency_of_peaks = 0
try:
    freeze_index = gp.freeze_of_gait(data[var])[1]
except:
    freeze_index = 0
try:
    freeze_count = sum(i > 2.0 for i in freeze_index)
except:
    freeze_count = 0
try:
    speed_of_gait = gp.speed_of_gait(data[var], wavelet_level = 6)
except:
    speed_of_gait = 0
        
### fill in values to each keys
feature_dict["no_of_steps_".format(var)] = no_of_steps
feature_dict["mean_freeze_index_".format(var)] = np.mean(freeze_index)
feature_dict["median_freeze_index_".format(var)] = np.median(freeze_index)
feature_dict["max_freeze_index_".format(var)] = np.max(freeze_index)
feature_dict["count_freeze_index_".format(var)] = freeze_count
feature_dict["speed_of_gait_".format(var)] = speed_of_gait
feature_dict["gait_step_regularity_".format(var)] = gait_step_regularity
feature_dict["gait_stride_regularity_".format(var)] = gait_stride_regularity
feature_dict["gait_symmetry_".format(var)] = gait_symmetry
feature_dict["frequency_of_peaks_".format(var)] = frequency_of_peaks
        
    ## Final Check to clear nan values to zero ##
for k, v in feature_dict.items():
    if np.isnan(v):
        feature_dict[k] = 0

Sampling rate is 99.33333333333333 Hz
Sampling rate is 99.33333333333333 Hz


In [59]:
feature_dict

{'no_of_steps_': 0,
 'mean_freeze_index_': 8.2914,
 'median_freeze_index_': 3.0395563,
 'max_freeze_index_': 117.14901,
 'count_freeze_index_': 36,
 'speed_of_gait_': 0.031323065007508595,
 'gait_step_regularity_': 0,
 'gait_stride_regularity_': 0,
 'gait_symmetry_': 0,
 'frequency_of_peaks_': 0.8908512228521506}

In [56]:
data

,timestamp,uptime,x,y,z,td,AA
time,,,,,,,
1970-01-01 00:00:00.000,0.086895,7177.766841,-0.065855,-0.918915,-0.401382,0.000000,1.003714
1970-01-01 00:00:00.010,0.096919,7177.776865,-0.044920,-0.910599,-0.397156,0.010024,0.993706
1970-01-01 00:00:00.020,0.106960,7177.786906,-0.034681,-0.906677,-0.394943,0.020065,0.989043
1970-01-01 00:00:00.030,0.117011,7177.796957,-0.030088,-0.906342,-0.393860,0.030116,0.988255
1970-01-01 00:00:00.040,0.127042,7177.806988,-0.053099,-0.917908,-0.397034,0.040147,1.000581
1970-01-01 00:00:00.050,0.137073,7177.817019,-0.066343,-0.921478,-0.402405,0.050178,1.006491
1970-01-01 00:00:00.060,0.147106,7177.827052,-0.064909,-0.918045,-0.398666,0.060211,1.001793
1970-01-01 00:00:00.070,0.157138,7177.837084,-0.040281,-0.910553,-0.395752,0.070243,0.993007
1970-01-01 00:00:00.080,0.167169,7177.847115,-0.036405,-0.907639,-0.392914,0.080274,0.989140
